# Przygotowanie danych do projektu rozpoznawania pisma ręcznego

Ten notebook wyjaśnia, jak przygotować dane do projektu rozpoznawania pisma ręcznego (HTR - Handwritten Text Recognition).

---

## **Informacje o danych**
- **Nazwa zbioru danych**: IAM Handwriting Database
- **Źródło danych**: [IAM Handwriting Database](http://www.fki.inf.unibe.ch/databases/iam-handwriting-database)
- **Warunki użytkowania**:
  - Dane są dostępne wyłącznie do celów badań niekomercyjnych.
  - Jeśli korzystasz z tych danych, musisz zarejestrować się na stronie projektu.
  - Przy publikacji pracy naukowej prosimy o odniesienie się do artykułu: *"The IAM-Database: An English Sentence Database for Off-line Handwriting Recognition"*.

---

## **Kroki**
1. Utworzenie struktury folderów.
2. Rozpakowanie zbioru danych.
3. Uruchomienie skryptów Python:
   - `create_labels.py` - Tworzy plik mapujący obrazy na teksty (`labels.json`).
   - `proccess_images.py` - Przetwarza obrazy na odpowiedni format i rozmiar.
   - `generate_split.py` - Generuje podział danych na zbiory treningowy, walidacyjny i testowy.
   - `split_data.py` - Przenosi dane do odpowiednich folderów na podstawie podziału.


---

# Tworzenie struktury katalogów

In [17]:
# Tworzenie struktury katalogów
import os

folders = [
    "../data/raw/",
    "../data/raw/ascii/",
    "../data/raw/lines/",
    "../data/processed/lines/",
    "../data/train/lines/",
    "../data/test/lines/",
    "../data/val/lines/",
    "../data/filtered/lines/"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)
    print(f"Utworzono folder: {folder}")


Utworzono folder: ../data/raw/
Utworzono folder: ../data/raw/ascii/
Utworzono folder: ../data/raw/lines/
Utworzono folder: ../data/processed/lines/
Utworzono folder: ../data/train/lines/
Utworzono folder: ../data/test/lines/
Utworzono folder: ../data/val/lines/
Utworzono folder: ../data/filtered/lines/


## Rozpakowanie zbioru danych

IAM Handwriting Database dostarcza dane w formie spakowanej (`.tgz`). Dlatego należy rozpakować pliki `ascii.tgz` i `lines.tgz` do odpowiednich folderów.


In [18]:
import tarfile

# Ścieżki do plików archiwum
ascii_archive = "../data/ascii.tgz"
lines_archive = "../data/lines.tgz"

# Rozpakowanie ascii.tgz
with tarfile.open(ascii_archive, "r:gz") as tar:
    tar.extractall(path="../data/raw/ascii/")
    print(f"Rozpakowano {ascii_archive}")

# Rozpakowanie lines.tgz
with tarfile.open(lines_archive, "r:gz") as tar:
    tar.extractall(path="../data/raw/lines/")
    print(f"Rozpakowano {lines_archive}")


Rozpakowano ../data/ascii.tgz
Rozpakowano ../data/lines.tgz


### Filtrowanie zdjęć oznaczonych jako "ok"

Zanim przejdziemy do kolejnych etapów przygotowania danych, upewnimy się, że wszystkie obrazy mają status `"ok"`. Zdjęcia oznaczone jako `"err"` zostaną wykluczone, ponieważ mogą zawierać błędy lub mieć niską jakość. Proces filtrowania działa na pliku `lines.txt` i przenosi odpowiednie obrazy do dedykowanego folderu.

In [19]:
!python ../src/filter_ok_images.py

Przeniesiono obrazy oznaczone jako 'ok' do folderu ../data/filtered/lines/


## Generowanie pliku `labels.json`

Teraz należy wykonać skrypt `create_labels.py`, który przekształci dane z pliku `lines.txt` w mapowanie plików obrazów na odpowiadające im teksty. Wynikowy plik zostanie zapisany jako `labels.json`.



In [20]:
!python ../src/create_labels.py


Plik labels.json został zapisany w ../data/processed/labels.json


## Przetwarzanie obrazów

Uruchomimy skrypt `proccess_images.py`, który:
1. Rekurencyjnie przejdzie przez wszystkie foldery w `data/raw/lines/`.
2. Przetworzy obrazy do skali szarości.
3. Zmieni rozmiar obrazów na `384x384`.
4. Zapisze wynik w folderze `data/processed/lines/`.


In [21]:
!python ../src/proccess_images.py


Maksymalna szerokość: 2260, Maksymalna wysokość: 267
Wszystkie obrazy zostały zapisane w ../data/processed/lines/


## Generowanie podziału na zbiory

Skrypt `generate_split.py` losowo podzieli przetworzone dane na:
- Zbiór treningowy (80% danych).
- Zbiór testowy (10% danych).
- Zbiór walidacyjny (10% danych).

Wynikowy podział zostanie zapisany w pliku `train-test-split.txt`.


In [22]:
!python ../src/generate_split.py


Podział danych zapisany w ../data/raw/ascii/train-test-split.txt


## Organizacja danych w katalogach

Na koniec należy uruchomić skrypt `split_data.py`, który:
1. Przeniesie dane zgodnie z plikiem `train-test-split.txt`.
2. Uporządkuje dane w folderach `train/`, `test/` i `val/`.


In [23]:
!python ../src/split_data.py


Podzielono dane na zbiory treningowy, walidacyjny i testowy.


# Podsumowanie

Dane powinny zostać poprawnie przygotowane:
- **Przetworzone obrazy**: `../data/processed/lines/`.
- **Zbiory danych**:
  - Zbiór treningowy: `../data/train/lines/`.
  - Zbiór testowy: `../data/test/lines/`.
  - Zbiór walidacyjny: `../data/val/lines/`.

